In [1]:
import os
#set paths for jwst pipeline
os.environ["CRDS_PATH"] = "/Users/mcha5804/JWST/crds_cache/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
from astropy.io import fits
from tqdm.auto import tqdm
from jwst.pipeline import Image2Pipeline
import rampitup as rm

os.chdir("/Users/mcha5804/JWST/data/jw01021005001_02101_00001_nis/")

In [8]:
from jwst.pipeline import Detector1Pipeline

2023-09-26 10:54:24,303 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2023-09-26 10:54:24,304 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2023-09-26 10:54:24,305 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2023-09-26 10:54:24,306 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2023-09-26 10:54:24,306 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2023-09-26 10:54:24,307 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2023-09-26 10:54:24,308 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2023-09-26 10:54:24,308 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2023-09-26 10:54:24,309 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2023-09-26 10:54:24,310 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2023-09-26 1

In [9]:
files = [
    "jw01021005001_gs-track_2022162142928_uncal.fits",
    # "uncal_abdor/jw01093015001_03102_00001_nis_uncal.fits",
    # "uncal_abdor/jw01093018001_03102_00001_nis_uncal.fits",
    ]

output_dir = "pipeline1_outputs"

for file in files:
    # pl1 = rm.Detector1Pipeline()
    pl1 = Detector1Pipeline()
    # pl1.save_results = True
    # pl1.output_dir = str(output_dir)
    # pl1.ipc.skip = True  
    # pl1.save_calibrated_ramp = True #save linearized ramp
    # #only run pipeline up to saving linearized ramps
    # pl1.lin_ramp_only = True
    # pl1.from_lin_ramp = False
    pl1.call(str(file))
    # pl1.run(str(file))

2023-09-25 11:35:57,307 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2023-09-25 11:35:57,308 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2023-09-25 11:35:57,309 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2023-09-25 11:35:57,310 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2023-09-25 11:35:57,311 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2023-09-25 11:35:57,312 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2023-09-25 11:35:57,312 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2023-09-25 11:35:57,313 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2023-09-25 11:35:57,314 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2023-09-25 11:35:57,315 - stpipe.Detector1Pipeline.lastframe - INFO - LastFrameStep instance created.
2023-09-25 1

CrdsLookupError: Error determining best reference for 'readnoise'  =    parameter='META.EXPOSURE.READPATT' value='TRACK' is not in ['FGS', 'FGS60', 'FGS8370', 'FGS840', 'FGSRAPID']

In [ ]:
#repeat for the other two observations
output_dir= "./pipeline1_outputs"

file_t = os.path.join(
    output_dir,
    "jw01093015001_03102_00001_nis_ramp.fits")

hdu = fits.open(file_t)
cube_t = hdu[1].data
cube_3t = hdu[3].data
cube_4t = hdu[4].data

for i in tqdm(range(1,cube_t.shape[1]+1)):

    hdu1 = fits.open(file_t)
    #insert data frames and metadata
    hdu1[1].data = cube_t[:,:i,:,:]
    hdu1[3].data = cube_3t[:,:i,:,:]
    hdu1[4].data = cube_4t[:,:i,:,:]
    
    output_dir2 = "./pipeline1_outputs_groups"

    file_t2 = os.path.join(
        output_dir2,
        "jw01093015001_03102_0000"+ str(i) +"_nis_uncal.fits")

    hdu1.writeto(file_t2,overwrite=False)
    hdu1.close()
    
    pl1 = rm.Detector1Pipeline()
    pl1.save_results = True
    pl1.output_dir = str(output_dir2)
    pl1.ipc.skip = True  
    pl1.save_calibrated_ramp = False 
    #run pipeline from linearized ramp step
    pl1.lin_ramp_only = False
    pl1.from_lin_ramp = True
    pl1.run(str(file_t2))
    
    pl2 = Image2Pipeline()
    pl2.save_results = True
    pl2.output_dir = output_dir2
    pl2.photom.skip = True
    pl2.resample.skip = True 
    fitsfile = file_t2.replace('_uncal.fits', '_rateints.fits')
    pl2.run(fitsfile)
    
    
    #actively delete files we don't need - probably a way to do this in the pipeline but oh well
    os.remove(file_t2) 
    file_t4 = os.path.join(
        output_dir2 ,
        "jw01093015001_03102_0000"+ str(i) +"_nis_rate.fits")
    os.remove(file_t4) 
    file_t5 = os.path.join(
        output_dir2 ,
        "jw01093015001_03102_0000"+ str(i) +"_nis_rateints.fits")
    os.remove(file_t5) 